<a href="https://colab.research.google.com/github/will-mccormack/CS-M148-Proj/blob/main/W3_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install scikit-lego

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 4.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import time
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [22]:
cleaned_data = pd.read_csv("https://raw.githubusercontent.com/will-mccormack/CS-M148-Proj/main/Data/train.csv")
validation_data = pd.read_csv("https://raw.githubusercontent.com/will-mccormack/CS-M148-Proj/main/Data/validation.csv")

In [6]:
numerical_cols = cleaned_data.select_dtypes(include=np.number).columns.tolist()
predictors = cleaned_data[numerical_cols].drop(columns=["popularity"]).columns.tolist()

model = sm.OLS(cleaned_data["popularity"],cleaned_data[predictors]).fit()
p_values = model.pvalues
p_values_threshold = 0.05
print(p_values >= p_values_threshold)

duration_ms          True
danceability        False
energy              False
key                  True
loudness            False
mode                False
speechiness         False
acousticness         True
instrumentalness    False
liveness            False
valence             False
tempo               False
time_signature      False
dtype: bool


In [17]:
# drop columns with p value over 0.05
drop_columns = p_values[p_values >= p_values_threshold].index.tolist()
cleaned_data = cleaned_data.drop(columns=drop_columns)
numerical_cols = cleaned_data.select_dtypes(include=np.number).columns.tolist()
predictors = cleaned_data[numerical_cols].drop(columns=["popularity"]).columns.tolist()

In [18]:
model = sm.OLS(cleaned_data["popularity"],cleaned_data[predictors]).fit()
p_values = model.pvalues
print(p_values >= p_values_threshold)

danceability        False
energy              False
loudness            False
mode                False
speechiness         False
instrumentalness    False
liveness            False
valence             False
tempo               False
time_signature      False
dtype: bool


check the model for training R2 vs validation R2 and MSE

In [26]:
predictions = model.predict(cleaned_data[predictors])
actual_predicted = pd.DataFrame({'Actual Popularity': cleaned_data['popularity'], 'Predicted Popularity': predictions})

px.scatter(actual_predicted, x='Actual Popularity', y='Predicted Popularity')

In [27]:
# Evaluate the model on the validation data
validation_predictions = model.predict(validation_data[predictors])

validation_r2 = r2_score(validation_data['popularity'], validation_predictions)
validation_mse = mean_squared_error(validation_data['popularity'], validation_predictions)

print(f"Validation R-squared: {validation_r2:.4f}")
print(f"Validation MSE: {validation_mse:.4f}")

Validation R-squared: 0.0205
Validation MSE: 486.8454


In [28]:
# Evaluate the model on the training data
train_r2 = model.rsquared
train_mse = mean_squared_error(cleaned_data['popularity'], predictions)

print(f"Training R-squared: {train_r2:.4f}")
print(f"Training MSE: {train_mse:.4f}")

Training R-squared: 0.6997
Training MSE: 488.6674


Training R2 is much higher than validation R2 which means there is high overfitting. We need to apply regularization